## Used cars database

**Over 370,000 used cars scraped from Ebay Kleinanzeigen.**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, RidgeCV, LassoCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
import bisect

## 1. Reading Data

In [ ]:
df = pd.read_csv("data/used-cars-database/autos.csv", sep=',', header=0, encoding='cp1252')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df = df[df["price"] > 0]
df = df[df["yearOfRegistration"] > 1950]
df = df[df["yearOfRegistration"] < 2020]
df = df[df["powerPS"] >0]
df = df[df["powerPS"] <1000]
df = df[df["price"] < 100000]

In [ ]:
df.head()

In [ ]:
registration_cols = ['yearOfRegistration', 'monthOfRegistration', 'dayOfRegistration']

df['monthOfRegistration'].replace(0, 1, inplace=True)
df['dayOfRegistration'] = 1

df['dateOfRegistration'] = df[registration_cols].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
df['dateOfRegistration'] = pd.to_datetime(df['dateOfRegistration'])

df.drop(registration_cols, axis=1, inplace=True)

In [ ]:
df['dateCrawled'] = pd.to_datetime(df['dateCrawled'])
df['dateCreated'] = pd.to_datetime(df['dateCreated'])
df['lastSeen'] = pd.to_datetime(df['lastSeen'])

## 2. Exploratory Data Analysis

### 2.1. Missing Values

In [ ]:
df.isna().sum()

In [ ]:
print(df.shape)

In [ ]:
threshold = 0.3
# Dropping columns with missing value rate higher than threshold in the training set
df = df[df.columns[df.isnull().mean() < threshold]]

# Dropping rows with missing value rate higher than threshold
df = df.loc[df.isnull().mean(axis=1) < threshold]

In [ ]:
print(df.shape)

In [ ]:
df.fillna(value="<UNK>", inplace=True)

### 2.2. Distribution of the target variable 

In [ ]:
plt.figure(figsize=(20,8))

plt.subplot(1,2,1)
plt.title('Car Price Distribution Plot')
sns.distplot(df.price)

plt.subplot(1,2,2)
plt.title('Car Price Spread')
sns.boxplot(y=df.price)

plt.show()

### 2.3. Numerical Variables

In [ ]:
num_cols = ['powerPS', 'kilometer']
fig_rows = np.ceil(len(num_cols)/2)

plt.figure(figsize=(15, fig_rows*5))
plt.subplots_adjust(hspace=0.5)

for ind, col in enumerate(num_cols):
    plt.subplot(fig_rows,2,ind+1)
    plt.title(col+' Distribution')
    ax = sns.distplot(df[col])
    ax.set(xlabel = col, ylabel='Frequency')

In [ ]:
num_cols = ['powerPS', 'kilometer']
fig_rows = np.ceil(len(num_cols)/2)

plt.figure(figsize=(15, fig_rows*5))
plt.subplots_adjust(hspace=0.5)

for ind, col in enumerate(num_cols):
    plt.subplot(fig_rows,2,ind+1)
    plt.title('price vs. '+col+' Scatter')
    ax = sns.scatterplot(x=df[col],y=df["price"])
    ax.set(xlabel = col, ylabel='price')

### 2.4. Catagorical Variables

In [ ]:
cat_cols =['seller', 'offerType', 'abtest', 'vehicleType', 'gearbox', 'model', 'fuelType', 'brand', 'notRepairedDamage']
fig_rows = np.ceil(len(cat_cols)/2)

plt.figure(figsize=(15, fig_rows*6))
plt.subplots_adjust(hspace=0.7)

for ind, col in enumerate(cat_cols):
    plt.subplot(fig_rows,2,ind+1)
    plt.title(col+' Histogram')
    ax = sns.countplot(df[col])
    ax.set(xlabel = col, ylabel='Frequency')
    xtickslabels=ax.get_xticklabels()
    ax.set_xticklabels(xtickslabels, rotation=80)
    # reduce number of ticks when too many
    if(len(xtickslabels)>20):
        for ind, label in enumerate(xtickslabels):
            if ind % 7 == 0:  # every 10th label is kept
                label.set_visible(True)
            else:
                label.set_visible(False)

In [ ]:
df.seller.value_counts()

In [ ]:
df.offerType.value_counts()

In [ ]:
df.name.nunique()

In [ ]:
df.model.nunique()

In [ ]:
df.postalCode.nunique()

In [ ]:
df.nrOfPictures.nunique()

In [ ]:
cat_cols =['seller', 'offerType', 'abtest', 'vehicleType', 'gearbox', 'model', 'fuelType', 'brand', 'notRepairedDamage']
fig_rows = np.ceil(len(cat_cols)/2)

plt.figure(figsize=(15, fig_rows*6))
plt.subplots_adjust(hspace=0.7)

for ind, col in enumerate(cat_cols):
    plt.subplot(fig_rows,2,ind+1)
    plt.title('price vs ' +col+' Boxplot')
    ax = sns.boxplot(x=df[col], y=df["price"])
    ax.set(xlabel = col, ylabel='price')
    xtickslabels=ax.get_xticklabels()
    ax.set_xticklabels(xtickslabels, rotation=80)
    # reduce number of ticks when too many
    if(len(xtickslabels)>20):
        for ind, label in enumerate(xtickslabels):
            if ind % 5 == 0:  # every 10th label is kept
                label.set_visible(True)
            else:
                label.set_visible(False)

### 2.5. Dates

In [ ]:
dates_cols = ['dateCrawled', 'dateCreated', 'lastSeen', 'dateOfRegistration']
fig_rows = np.ceil(len(cat_cols)/2)

plt.figure(figsize=(15, fig_rows*5))
plt.subplots_adjust(hspace=0.5)

for ind, col in enumerate(dates_cols):
    plt.subplot(fig_rows,2,ind+1)
    plt.title(col+' Histogram')
    ax = sns.countplot((df[col].dt.year.astype(str) + '-' + df[col].dt.month.astype(str)).sort_values())
    ax.set(xlabel = col, ylabel='Frequency')
    xtickslabels=ax.get_xticklabels()
    ax.set_xticklabels(xtickslabels, rotation=80)
    # reduce number of ticks when too many
    if(len(xtickslabels)>20):
        for ind, label in enumerate(xtickslabels):
            if ind % 20 == 0:  # every 10th label is kept
                label.set_visible(True)
            else:
                label.set_visible(False)

## 3. Data Splitting

In [ ]:
y = df.pop("price")
X = df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=8)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

## 4. Feature Engineering

### 4.1. Removing non-predictive features

In [ ]:
X_train.drop(['dateCrawled', 'name', 'seller', 'offerType', 'nrOfPictures', 'postalCode', 'lastSeen'], axis=1, inplace=True)
X_test.drop(['dateCrawled', 'name', 'seller', 'offerType', 'nrOfPictures', 'postalCode', 'lastSeen'], axis=1, inplace=True)

### 4.2. Create new features

#### Adding feature 'vehicle age'

In [ ]:
X_train['age'] = (X_train['dateCreated'] -  X_train['dateOfRegistration'])/ np.timedelta64(1, 'Y')
X_test['age'] = (X_test['dateCreated'] -  X_test['dateOfRegistration'])/ np.timedelta64(1, 'Y')

In [ ]:
plt.figure(figsize=(10, 7))

plt.title('age Distribution')
ax = sns.distplot(X_train['age'])
ax.set(xlabel = 'age', ylabel='Frequency')

In [ ]:
X_train.drop('dateCreated', axis=1, inplace=True)
X_test.drop('dateCreated', axis=1, inplace=True)
X_train.drop('dateOfRegistration', axis=1, inplace=True)
X_test.drop('dateOfRegistration', axis=1, inplace=True)

#### Creating polynomial features for numerical columns

In [ ]:
num_cols = ['kilometer', 'powerPS', 'age']

In [ ]:
PF = PolynomialFeatures(degree=4, include_bias=False)

# Encode categorical columns, and store results in a new dataframe
X_train_poly = pd.DataFrame(PF.fit_transform(X_train[num_cols]), index=X_train.index)
X_test_poly = pd.DataFrame(PF.transform(X_test[num_cols]), index=X_test.index)

# Adding poly columns to initial datasets
poly_feature_names = PF.get_feature_names()
replace_dict = {"x"+str(i):num_cols[i] for i in range(3)}

for ind, name in replace_dict.items():
    poly_feature_names = [w.replace(ind, name) for w in poly_feature_names]
X_train_poly.columns = poly_feature_names
X_test_poly.columns = poly_feature_names

# Drop initial columns 
X_train.drop(num_cols ,axis=1, inplace=True)
X_test.drop(num_cols ,axis=1, inplace=True)
num_cols = poly_feature_names

# Add feature engineered columns to initial dataset
X_train = pd.concat([X_train, X_train_poly], axis=1)
X_test = pd.concat([X_test, X_test_poly ], axis=1) 

In [ ]:
#num_cols = ['mileage', 'engine_power', 'age']

#for col in num_cols:
#    X_train[col+'2'] = X_train[col]**2
#    X_test[col+'2'] = X_test[col]**2
#    X_train[col+'3'] = X_train[col]**3
#    X_test[col+'3'] = X_test[col]**3

### 4.3. Scaling numerical variables

In [ ]:
SS = StandardScaler()
scaled_cols = num_cols 

X_train[scaled_cols] = SS.fit_transform(X_train[scaled_cols])
X_test[scaled_cols] = SS.transform(X_test[scaled_cols])

In [ ]:
X_train.dtypes

In [ ]:
X_train.head()

### 4.4. Label encoding

In [ ]:
set(X_test['brand'])-set(X_train['brand'])

In [ ]:
set(X_test['model'])-set(X_train['model'])

In [ ]:
LE = LabelEncoder()
le_cols = [] 

for col in le_cols:
    # fit transform the training set
    X_train[col] = LE.fit_transform(X_train[col])

    # transform the test set
    X_test[col] = X_test[col].map(lambda s: 'other' if s not in LE.classes_ else s)
    le_classes = LE.classes_.tolist()
    bisect.insort_left(le_classes, 'other')
    LE.classes_ = le_classes

    X_test[col] = LE.transform(X_test[col])

### 4.5. One hot encoding of other catagorical features

In [ ]:
OHE = OneHotEncoder(drop = None, handle_unknown='ignore', sparse=False)

# Define categorical to be one hot encoded
ohe_cols = ["abtest", "vehicleType", "gearbox", "model", "fuelType", "brand", "notRepairedDamage"]

# Encode categorical columns, and store results in a new dataframe
X_train_encoded = pd.DataFrame(OHE.fit_transform(X_train[ohe_cols]), index=X_train.index)
X_test_encoded = pd.DataFrame(OHE.transform(X_test[ohe_cols]), index=X_test.index)

# Retrieve encoded columns names
X_train_encoded.columns = OHE.get_feature_names(ohe_cols)
X_test_encoded.columns = OHE.get_feature_names(ohe_cols)

# Drop initial columns 
X_train.drop(ohe_cols ,axis=1, inplace=True)
X_test.drop(ohe_cols ,axis=1, inplace=True)

# Add encoded columns to initial dataset
X_train = pd.concat([X_train, X_train_encoded], axis=1)
X_test = pd.concat([X_test, X_test_encoded ], axis=1)

## 5. Linear Regression

In [ ]:
# fit linear model
lr = LinearRegression()
lr.fit(X_train, y_train)

# predicting over training & testing datasets
y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)

In [ ]:
rmse = (np.sqrt(mean_squared_error(y_train, y_train_pred)))
r2 = r2_score(y_train, y_train_pred)

print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

# model evaluation for testing set
rmse = (np.sqrt(mean_squared_error(y_test, y_test_pred)))
r2 = r2_score(y_test, y_test_pred)

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))

## 6. Ridge

In [ ]:
ridge = RidgeCV(alphas=[10**i for i in np.arange(-4, 4,  0.2, dtype=float)], scoring='r2', cv=5)

ridge.fit(X_train, y_train)

# predicting over training & testing datasets
y_train_pred = ridge.predict(X_train)
y_test_pred = ridge.predict(X_test)

This is equivalent to:

In [ ]:
## training the model
param_grid = {'alpha': [10**i for i in np.arange(-4, 4,  0.2, dtype=float)]}

ridge = Ridge(normalize=False)
ridge_search = GridSearchCV(ridge, param_grid, scoring='r2', cv=5)
ridge_search.fit(X_train, y_train)

# predicting over training & testing datasets
y_train_pred = ridge_search.predict(X_train)
y_test_pred = ridge_search.predict(X_test)

In [ ]:
ridge_search.best_estimator_

In [ ]:
# model evaluation for training set
rmse = (np.sqrt(mean_squared_error(y_train, y_train_pred)))
r2 = r2_score(y_train, y_train_pred)

print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

# model evaluation for testing set
rmse = (np.sqrt(mean_squared_error(y_test, y_test_pred)))
r2 = r2_score(y_test, y_test_pred)

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))

## 7. Lasso

In [ ]:
## training the model
param_grid = {'alpha': [10**i for i in np.arange(-4, 4,  0.5, dtype=float)]}

lasso = Lasso(normalize=False)
lasso_search = GridSearchCV(lasso, param_grid, scoring='r2', cv=5)
lasso_search.fit(X_train, y_train)

# predicting over training & testing datasets
y_train_pred = lasso_search.predict(X_train)
y_test_pred = lasso_search.predict(X_test)

In [ ]:
lasso_search.best_estimator_

In [ ]:
# model evaluation for training set
rmse = (np.sqrt(mean_squared_error(y_train, y_train_pred)))
r2 = r2_score(y_train, y_train_pred)

print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

# model evaluation for testing set
rmse = (np.sqrt(mean_squared_error(y_test, y_test_pred)))
r2 = r2_score(y_test, y_test_pred)

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))

## 9. Random Forest Regressor

In [ ]:
[int(x) for x in np.linspace(start = 100, stop = 400, num = 4)]

In [ ]:
# Create the random grid
random_grid = { # Number of trees in random forest
               'n_estimators': [int(x) for x in np.linspace(start = 100, stop = 400, num = 4)],
                # Number of features to consider at every split
               'max_features': ['auto', 'sqrt', 'log2'],
                # Maximum number of levels in tree
               'max_depth': [int(x) for x in np.linspace(10, 110, num = 11)] + [None],
                # Minimum number of samples required to split a node
               'min_samples_split': [2, 5, 10],
                # Minimum number of samples required at each leaf node
               'min_samples_leaf': [1, 2, 4],
                # Method of selecting samples for training each tree
               'bootstrap': [True, False]
              }

# Create the random grid
random_grid = {
               'n_estimators': [400],
               'max_features': ['auto', 'sqrt', 'log2'],
               'max_depth': [5, 10, 30] + [None]
              }

print(random_grid)
rf = RandomForestRegressor(criterion="mse", random_state=0)
rf_search = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 12, cv = 3, verbose=2, random_state=0)
rf_search = rf_search.fit(X_train, y_train)

In [ ]:
rf_search.best_estimator_

In [ ]:
# predicting over training & testing datasets
y_train_pred = rf_search.predict(X_train)
y_test_pred = rf_search.predict(X_test)

In [ ]:
# model evaluation for training set
rmse = (np.sqrt(mean_squared_error(y_train, y_train_pred)))
r2 = r2_score(y_train, y_train_pred)

print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

# model evaluation for testing set
rmse = (np.sqrt(mean_squared_error(y_test, y_test_pred)))
r2 = r2_score(y_test, y_test_pred)

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))

In [ ]:
predictions = pd.DataFrame({"Actual": y_test, "Predicted": y_test_pred})
# plot a sample of observations
predictions = predictions.sample(n=200)
predictions = predictions.sort_index().reset_index()

plt.figure(figsize=(18, 8))
predictions['Actual'].plot(legend=True)
predictions['Predicted'].plot(legend=True)
plt.title('Actual vs. Predicted Price')
plt.ylabel('Price')
plt.show()

## 10. Gradient Boost Regressor

In [ ]:
# Create the random grid
random_grid = { # Number of trees in random forest
               'n_estimators': [int(x) for x in np.linspace(start = 50, stop = 400, num = 8)],
                # Number of features to consider at every split
               'max_features': ['auto', 'sqrt', 'log2'],
                # Maximum number of levels in tree
               'max_depth': [int(x) for x in np.linspace(10, 110, num = 11)] + [None],
                # Minimum number of samples required to split a node
               'min_samples_split': [2, 5, 10],
                # Minimum number of samples required at each leaf node
               'min_samples_leaf': [1, 2, 4],
               'learning_rate': [0.1, 1, 10],
               'subsample': [0.5, 0.7, 0.9]
             }

# Create the random grid
random_grid = {
               'n_estimators': [500],
               'max_features': ['auto', 'sqrt', 'log2'],
               'max_depth': [5, 10, 30] + [None],
               'subsample': [0.7, 1]
              }

print(random_grid)
gb = GradientBoostingRegressor(random_state=0)
gb_search = RandomizedSearchCV(estimator = gb, param_distributions = random_grid, n_iter = 12, cv = 3, verbose=2, random_state=0)
gb_search = gb_search.fit(X_train, y_train)

In [ ]:
gb_search.best_params_

In [ ]:
# predicting over training & testing datasets
y_train_pred = gb_search.predict(X_train)
y_test_pred = gb_search.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# model evaluation for training set
rmse = (np.sqrt(mean_squared_error(y_train, y_train_pred)))
r2 = r2_score(y_train, y_train_pred)

print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

# model evaluation for testing set
rmse = (np.sqrt(mean_squared_error(y_test, y_test_pred)))
r2 = r2_score(y_test, y_test_pred)

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))